mmdetectron uses torch.utils.data.DataLoader class to create dataloader.  
In config `samples_per_gpu` is batchsize and `workers_per_gpu` is num_workers.  


In [3]:
import json
from mmdet.datasets import CocoDataset
from mmdet.datasets.pipelines import LoadImageFromFile, LoadAnnotations, RandomFlip, Normalize, DefaultFormatBundle, Collect

with open('small_coco/instances_train2017_small.json', 'r') as f:
    data = json.load(f)

dataset = CocoDataset(classes=[cat['name'] for cat in data['categories']],
                      ann_file ='small_coco/instances_train2017_small.json',
                      img_prefix='small_coco/train_2017_small',
                      pipeline = [LoadImageFromFile(),
                                  LoadAnnotations(),
                                  RandomFlip(flip_ratio=0.5),
                                  Normalize(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True),
                                  DefaultFormatBundle(),
                                  Collect(keys=['img', 'gt_bboxes', 'gt_labels'])])

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


### Sampler

samplers used are GroupSampler or DistributedGroupSampler for Training and DistributedSampler for validation/test

In [32]:
from mmdet.datasets.samplers import GroupSampler

sampler = GroupSampler(dataset, samples_per_gpu=4)
for x in sampler:
    print(x, end=' ')

13 5 4 14 9 6 0 7 10 3 14 10 11 8 15 2 12 1 7 11 

### Collate function

In [36]:
dataset[0]

{'img_metas': DataContainer({'filename': 'small_coco/train_2017_small/000000391895.jpg', 'ori_filename': '000000391895.jpg', 'ori_shape': (360, 640, 3), 'img_shape': (360, 640, 3), 'pad_shape': (360, 640, 3), 'scale_factor': 1.0, 'flip': True, 'flip_direction': 'horizontal', 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}),
 'img': DataContainer(tensor([[[ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          ...,
          [-1.4329, -1.1760, -1.1760,  ...,  0.8276,  0.1254,  0.6906],
          [-0.9877, -0.8678, -0.8849,  ...,  0.4508,  0.3138,  1.3242],
          [-1.0904, -1.1760, -1.3130,  ...,  1.1015,  1.3242,  0.8961]],
 
         [[ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2

In [49]:
from mmcv.parallel import DataContainer, collate
from functools import partial

if stack=False, cpu_only=True ('img_metas')

In [56]:
dcs = [DataContainer({'n':i}, cpu_only=True) for i in range(4)]
print(dcs)
collate(dcs, 2)

[DataContainer({'n': 0}), DataContainer({'n': 1}), DataContainer({'n': 2}), DataContainer({'n': 3})]


DataContainer([[{'n': 0}, {'n': 1}], [{'n': 2}, {'n': 3}]])

if stack=True, cpu_only=False ('img')

In [88]:
import torch
dcs = [DataContainer(torch.randn(3, i, i), stack=True) for i in range(1, 5)]
print(tuple([dc.data.shape] for dc in dcs))
collated = collate(dcs, 2)
print([d.shape for d in collated.data])

([torch.Size([3, 1, 1])], [torch.Size([3, 2, 2])], [torch.Size([3, 3, 3])], [torch.Size([3, 4, 4])])
[torch.Size([2, 3, 2, 2]), torch.Size([2, 3, 4, 4])]


if stack=False, cpu_only=False ('gt_bboxes')

In [92]:
import torch
dcs = [DataContainer(torch.randn(3, i, i)) for i in range(1, 5)]
print([[dc.data.shape] for dc in dcs])
collated = collate(dcs, 2)
print([[data.shape for data in l] for l in collated.data])

[[torch.Size([3, 1, 1])], [torch.Size([3, 2, 2])], [torch.Size([3, 3, 3])], [torch.Size([3, 4, 4])]]
[[torch.Size([3, 1, 1]), torch.Size([3, 2, 2])], [torch.Size([3, 3, 3]), torch.Size([3, 4, 4])]]


In [8]:
from torch.utils.data import DataLoader

In [59]:
data_loader = DataLoader(
        dataset,
        batch_size=4,
        sampler=sampler,
        num_workers=2,
        collate_fn=partial(collate, samples_per_gpu=2))

In [60]:
next(iter(data_loader))

{'img_metas': DataContainer([[{'filename': 'small_coco/train_2017_small/000000005802.jpg', 'ori_filename': '000000005802.jpg', 'ori_shape': (479, 640, 3), 'img_shape': (479, 640, 3), 'pad_shape': (479, 640, 3), 'scale_factor': 1.0, 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}, {'filename': 'small_coco/train_2017_small/000000184613.jpg', 'ori_filename': '000000184613.jpg', 'ori_shape': (336, 500, 3), 'img_shape': (336, 500, 3), 'pad_shape': (336, 500, 3), 'scale_factor': 1.0, 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}], [{'filename': 'small_coco/train_2017_small/000000060623.jpg', 'ori_filename': '000000060623.jpg', 'ori_shape': (427, 640, 3), 'img_shape': (427, 640, 3), 'pad_shape': (427, 640, 3), 'scale_

this can be simplified with `build_dataloader`

In [131]:
from mmdet.datasets import build_dataloader

dataloader = build_dataloader(dataset, samples_per_gpu=4, workers_per_gpu=2)
next(iter(dataloader))

{'img_metas': DataContainer([[{'filename': 'small_coco/train_2017_small/000000060623.jpg', 'ori_filename': '000000060623.jpg', 'ori_shape': (427, 640, 3), 'img_shape': (427, 640, 3), 'pad_shape': (427, 640, 3), 'scale_factor': 1.0, 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}, {'filename': 'small_coco/train_2017_small/000000222564.jpg', 'ori_filename': '000000222564.jpg', 'ori_shape': (480, 640, 3), 'img_shape': (480, 640, 3), 'pad_shape': (480, 640, 3), 'scale_factor': 1.0, 'flip': True, 'flip_direction': 'horizontal', 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}, {'filename': 'small_coco/train_2017_small/000000522418.jpg', 'ori_filename': '000000522418.jpg', 'ori_shape': (480, 640, 3), 'img_shape': (480, 640, 3), 'pad_shape': (480, 640, 3), 's